# SAMap zebrafish Farrell vs amphioxus preprocess
With leiden-based neighborhoods

In [ ]:
from samap.mapping import SAMAP
from samap.analysis import (get_mapping_scores, GenePairFinder, sankey_plot, chord_plot, CellTypeTriangles, 
                            ParalogSubstitutions, FunctionalEnrichment,
                            convert_eggnog_to_homologs, GeneTriangles)
from samalg import SAM
import pandas as pd
import numpy
import seaborn as sns
import csv
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
import scanpy as sc
import warnings
warnings.filterwarnings(action='ignore')
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

## Read parameters

In [ ]:
with open('config.csv') as f:
    config=[tuple(line) for line in csv.reader(f)]

config=dict(config)
contrast='farrell_'+config["stage_zebra"]+'_vs_'+config["stage_amp"]
print(contrast)

Show parameters.

In [ ]:
pd.DataFrame.from_dict(config,orient='index',dtype='str')

## Read gene name mapping

Parse mapping of transcript ids vs gene names used in the anndata object

In [ ]:
with open('../../../reference/zebra_ens_prot/zf_final_prot_gene_map.csv') as f:
    dr_idmap=[tuple(line) for line in csv.reader(f)][2:]

with open('../../../reference/amp_braflo100_cdna/amp_tr_gene_map.csv') as f:
    bf_idmap=[tuple(line) for line in csv.reader(f)][2:]

print(dr_idmap[:5])
print(bf_idmap[:5])

# Running SAMap

## Loading in raw data

In [ ]:
path_dr = config["path_zebra"]
path_amp = config["path_amp"]

Set input data path.

In [ ]:
filenames = {'dr':path_dr,'bf':path_amp}

Instantiate the SAMAP object.

In [ ]:
sm = SAMAP(
        filenames,
        f_maps = '../../../reference/maps_prot/maps_ens/',
        names = {'dr' : dr_idmap, 'bf' : bf_idmap},
        save_processed=False
    )

Show unintegrated umaps.

In [ ]:
sc.pl.embedding(sm.sams['bf'].adata,
                    basis='X_umap',
                    color='celltype_coarse2',
                title=('Amphioxus unintegrated '+config["stage_amp"]),
                    frameon=False)

sc.pl.embedding(sm.sams['dr'].adata,
                    basis='X_umap',
                title=('Zebrafish unintegrated '+config["stage_zebra"]),
                    color='celltype',
                    frameon=False)

## Run SAMap

In [ ]:
sm.run(ncpus=16)

Save samap object.

In [ ]:
save_object(sm,'sam_processed.pkl')

## Visualizing the combined projection

In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(5)
fig.set_figheight(5)
fig.tight_layout()
ax.set_title('SAMap UMAP amphioxus '+config["stage_amp"]+' (red) vs Zebrafish '+config["stage_zebra"]+' (blue)')
sm.scatter(COLORS={'dr':'#10b9f1','bf':'#f14810'},axes=ax,ss={'bf':5,'dr':5},alpha=0.5)